In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score, recall_score, accuracy_score, recall_score, f1_score, precision_score, roc_auc_score 
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
#import matplotlib.patches as mpatches
import numpy as np
import scipy.io as sp
import pandas as pd
#from sklearn.decomposition import PCA
import xlsxwriter
from IPython.display import display
from sklearn.model_selection import StratifiedKFold

#Traitement du fichier .mat
cancercell = sp.loadmat('breastw.mat')
X = cancercell['X']
y = cancercell['y']
#print(cancercell)
y=np.reshape(y,(683,))



In [2]:
#Choix hyperparamètres 

HLS=[]
for j in range(2,10):
    for k in range(2,10):
        HLS.append((j,k))
Alpha = [1, 0.1, 0.01, 0.001, 0.0001]
Solver = ['lbfgs', 'sgd','adam']
Activation = ['identity', 'logistic', 'tanh', 'relu']
Max_iter = []
for k in range(3,10):
    Max_iter.append(2*k)

   
skf = StratifiedKFold(n_splits=10,shuffle=True) # équilibrage dataset


In [ ]:
for train_index, test_index in skf.split(X,y):
    
    test =False
    X_train, X_test = X[train_index,:], X[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    y_train=np.ravel(y_train)
    y_test=np.ravel(y_test)

    #Méthode MLP

    test =False
    
    for k in HLS :
        for i in Alpha :
            for j in Solver :
                for p in Activation : 
                    for n in Max_iter :
                        
                        #algo MLP
                        model_MLP = MLPClassifier(hidden_layer_sizes = k, alpha = i, solver = j, activation = p, max_iter = n)
                        model_MLP.fit(X_train,y_train)
                        y_pred_MLP = model_MLP.predict(X_test)
                        
                        y_scores_MLP = model_MLP.predict_proba(X_test)
                        y_scores_MLP = y_scores_MLP[:, 1]
                        
                        F1_MLP = round(f1_score(y_test,y_pred_MLP),3)
                        recall_MLP = round(recall_score(y_test,y_pred_MLP),3)
                        precision_MLP = round(precision_score(y_test,y_pred_MLP,zero_division=0),3)
                        accuracy_MLP = round(accuracy_score(y_test,y_pred_MLP),3)
                    
                        #data = np.append(data, [[k,i,j,p,n,F1_MLP,recall_MLP,precision_MLP,accuracy_MLP,AUC_MLP]],axis=0)
                        if test == False :
                            data = np.array([[k,i,j,p,n,F1_MLP,recall_MLP,precision_MLP,accuracy_MLP]])
                            test = True
                        else :
                            data = np.append(data,np.array([k,i,j,p,n,F1_MLP,recall_MLP,precision_MLP,accuracy_MLP]).reshape(1,9),axis=0)
                        #Fin méthode MLP


In [ ]:
#Creation DataFrame

df = pd.DataFrame(data, columns=['Hidden_layers','Alpha','solver','Activation','Max_iter','F1 score','recall','precision','accuracy'])
df = df.astype({'F1 score': float, 'Recall': float,'Precision': float,'Accuracy': float})

filtered_values_1 = np.where((df['F1 score']>=0.96) & (df['Recall']>=0.925) & (df['Precision']>=0.925))
display(df.iloc[filtered_values_1].sort_values(by=['F1 score'], ascending=False))
